In [1]:
rm(list=ls())

In [2]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,609084,32.6,1319145,70.5,1305148,69.8
Vcells,1138183,8.7,8388608,64.0,1821133,13.9


In [3]:
# install.packages("tidyverse") # sagemaker has
library(tidyverse)
# install.packages("parallel") # sagemaker has
library(parallel)
# install.packages("cluster") # sagemaker has
library(cluster)

install.packages("forecast") # sagemaker doesn't have
library(forecast)

install.packages("RcppArmadillo") # sagemaker doesn't have
library(RcppArmadillo)

install.packages("Metrics") # sagemaker doesn't have
library(Metrics)

install.packages("biglm") # sagemaker doesn't have
library(biglm)

install.packages("tseries") # sagemaker doesn't have
library(tseries)

install.packages("scoringutils") # sagemaker doesn't have
library(scoringutils)

Warning message:
“‘timedatectl’ indicates the non-existent timezone name ‘n/a’”
Warning message:
“Your system is mis-configured: ‘/etc/localtime’ is not a symlink”
Warning message:
“It is strongly recommended to set envionment variable TZ to ‘Etc/UCT’ (or equivalent)”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done


At

# Read In Data

In [4]:
clust_assign <- read_csv("Results/Unseen Sensor/clust_assign.csv")

New names:
• `` -> `...1`
Rows: 4 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (5): ...1, rand, catch22, tsfeat, dtw

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
fnames <- list.files("Data/Unseen Sensor/Processed/", pattern="*.csv", full.names=TRUE)
total_df_list <- lapply(fnames, read_csv)

Rows: 35040 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): site_name
dbl  (5): day_of_week, day_of_year, interval_of_day, avg_mph, total_volume
lgl  (2): missing_speed, missing_volume
dttm (1): timestamp
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 35040 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): site_name
dbl  (5): day_of_week, day_of_year, interval_of_day, avg_mph, total_volume
lgl  (2): missing_speed, missing_volume
dttm (1): timestamp
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 35040 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delim

In [6]:
total_df_list <- lapply(total_df_list, function(x) x %>% arrange(timestamp))    

In [7]:
# Randomly create a list of starting points from which we we sample each data frame

# Set the seed
set.seed(12345)

# Sample without replacement as to ensure the starting time for each time series is different
starting_points <- sample(1:((96*365)-(96*12*7)), 
                          length(total_df_list), 
                          replace=FALSE)

# Create the ending points by adding on 12 weeks to the starting points
ending_points <- starting_points + (96*7*12) - 1

In [8]:
write.csv(data.frame(start=starting_points,
                     end=ending_points),
          "unseen_sensor_start_end_points.csv"
          )

In [9]:
# For each df in our list, create a row number column called 'rn' - this will allow us to find the starting point
# for each sample using the above starting_points array
total_df_list <- lapply(total_df_list, function(x) x %>% mutate(rn = row_number()))

In [10]:
# Sample each df in the list according to its corresponding starting and ending point - this creates 12-week
# long samples of each data frame which will be used for modeling and testing
total_df_list_samples <- lapply(1:length(total_df_list), 
                                function(x) total_df_list[[x]] %>% 
                                    filter(rn >= starting_points[x]) %>%
                                    filter(rn <= ending_points[x]))

In [11]:
# Add a new column to each data frame to denote whether each row should be part of the training set, validation set
# or test set - the first 8 weeks are designated as train, the next 2 as val, and the final 2 as test. It may be 
# the case that train and val get used for training, depending on the modeling task
total_df_list_samples <- lapply(total_df_list_samples, 
                                function(x) x %>% mutate(rn = row_number()) %>%
                                    mutate(train_val_test = ifelse(rn <= (96*7*8), 
                                                                   "train", 
                                                                   ifelse(rn <= (96*7*10), 
                                                                          "val", 
                                                                          "test")))
                                
                               )

# Functions for Global AR

In [12]:
lag_embed <- function(data, lag=1) {
    # This function creates a lag emebedded matrix at the provided lag level. It assumes there is a train_val_test
    # in the input data frame, and returns a list of the training, validaiton, and test data as lag matrices
    
    # Compute the number of rows of test and validation data
    n_test <- nrow(data %>% filter(train_val_test=="test"))
    n_val <- nrow(data %>% filter(train_val_test=="val"))
    
    # Lag embed the entire time series - note we use the input lag+1 to account for the target variable
    embed_mat <- embed(data$total_volume, (lag+1))

    # Get the total numebr of rows in the lag matrix
    n_mat <- nrow(embed_mat)
    
    # Based on the number of rows in the matrix and the number of test and validation samples,
    # compute the indexes which correspond to test and validation
    test_idx <- rev(1:n_mat)[1:n_test]
    val_idx <- rev(1:n_mat)[-c(1:n_test)][1:n_val]
    
    # Remoe the test and val indexes to get the training matrix
    train_mat <- embed_mat[-c(val_idx, test_idx),]
    
    # Filter using the above computed indexes to get the validaiton and test matrices
    val_mat <- embed_mat[val_idx,]
    test_mat <- embed_mat[test_idx,]
    
    # Return a list of the trianing, validation, and test lag-embedded matrices
    return_list <- list("train"=train_mat, "val"=val_mat, "test"=test_mat)
}

In [13]:
global_ar_least_squares_pred <- function(model,
                                         test_list,
                                         return_preds=TRUE,
                                         return_means=TRUE
                                        ) 
{
    # Function which takes in a model and a list of test matrices and makes predictions 
    # and computes performance measurements on that test data
    
    # Rename the model object
    big_lm_mod <- model
    
    # Initialize an empty df for storing the predictions
    test_preds <- data.frame()
    # For each matrix in the test list
    for (test_t in 1:length(test_list)) {
        # Compute the one-step ahead predictions
        test_pred <- data.frame(forec=predict(big_lm_mod, newdata=test_list[[test_t]]), ts_index=test_t)
        # Append those to the test_pred df
        test_preds <- rbind(test_preds, test_pred)
    }

    # Initialize empty lists for rmse, mae, and means
    test_rmse_ls <- c()
    test_mae_ls <- c()
    test_means <- c()
    
    # Loop through each time series in the data set and
    for (idx in 1:length(test_list)) {
        
        # Compute the mean and append to the list of means
        test_mean_i <- mean(test_list[[idx]][,1])
        test_means <- c(test_means, test_mean_i)
        
        # Compute the prediction RMSE and append to the list of RMSE
        rmse_test_i <- rmse(test_list[[idx]][,1], (test_preds %>% filter(ts_index==idx))$forec)
        test_rmse_ls <- c(test_rmse_ls, rmse_test_i)
    
        # Copmute the MAE of the predictions and append to list
        mae_test_i <- mae(test_list[[idx]][,1], (test_preds %>% filter(ts_index==idx))$forec)
        test_mae_ls <- c(test_mae_ls, mae_test_i)
    }
    
    # Set up the return_list to include the rmse and mae lists
    return_list <- list("test_rmse"=test_rmse_ls, "test_mae"=test_mae_ls)
    
    # If the user specifies, return the test set predicitons and the target means as well
    if (return_preds) {
        return_list <- append(return_list, list("test_preds"=test_preds))
    }
    
    if (return_means) {
        return_list <- append(return_list, list("test_means"=test_means))
    }
    
    # Return the return_list
    return_list        
}

In [14]:
bootstrap_pred_int <- function(predictions,
                               residuals,
                               n_boot,
                               actuals=NULL
                              ) 
{
    # Function to compute prediction intervals using bootstrapping with model residuals
    # function takes in the predicted values, the model residuals, the number of bootstrap samples to use,
    # and an optional set of true values (used for computing prediction interval scores)
    
    # Intialize empty lists for the hi and lo components of the 80% nad 95% PIs
    boot_lo_80 <- c()
    boot_hi_80 <- c()
    boot_lo_95 <- c()
    boot_hi_95 <- c()

    # Loop through every row in the predictions
    for (i in 1:nrow(predictions)){
        # Reset the seed to the loop counter - could make this more random, but this is for reproducibility
        set.seed(i)
        
        # Sample with replacement from the residuals n_boot times
        boot_samples <- sample(residuals, n_boot, replace=TRUE)
        # Add the predicted value to each sample
        boot_preds <- boot_samples + predictions$forec[i]
        
        # Compute the PIs as the quantiles of these pred+boot values
        boot_pred_lo_80 <- quantile(boot_preds, 0.1)
        boot_pred_hi_80 <- quantile(boot_preds, 0.9)

        boot_pred_lo_95 <- quantile(boot_preds, 0.025)
        boot_pred_hi_95 <- quantile(boot_preds, 0.975)
        
        # Append the PI values to the lists
        boot_lo_80 <- c(boot_lo_80, boot_pred_lo_80)
        boot_hi_80 <- c(boot_hi_80, boot_pred_hi_80)

        boot_lo_95 <- c(boot_lo_95, boot_pred_lo_95)
        boot_hi_95 <- c(boot_hi_95, boot_pred_hi_95)        
    }
    
    # Create a copy of the input df called predicitons
    pred_w_boot_int_df <- predictions
    
    # Add columns to the df for the PIs
    pred_w_boot_int_df$lo_80 <- boot_lo_80
    pred_w_boot_int_df$hi_80 <- boot_hi_80
    pred_w_boot_int_df$lo_95 <- boot_lo_95
    pred_w_boot_int_df$hi_95 <- boot_hi_95
    
    # If the true values are included
    if (!is.null(actuals)){
        
        # Add a column for the true values
        pred_w_boot_int_df$true_values <- actuals
        
        # Compute the 80% and 95% interval scores and add them as columns in the df
        pred_w_boot_int_df <- pred_w_boot_int_df %>%
            mutate(int_score_80 = interval_score(true_values=true_values,
                                                 lower=lo_80,
                                                 upper=hi_80,
                                                 interval_range=80,
                                                 weigh=FALSE
                                                )) %>% 
            mutate(int_score_95 = interval_score(true_values=true_values,
                                                 lower=lo_95,
                                                 upper=hi_95,
                                                 interval_range=95,
                                                 weigh=FALSE
                                                ))
    }
    
    # Return the final set of preds with PIs (and interval scores if applicable)
    pred_w_boot_int_df
    
}

# Global AR Model for Full Data Set

In [15]:
# Create the final lag embedd matrix list using the lag of 840
embed_mat_list <- lapply(total_df_list_samples,
                     function(x) lag_embed(x, lag=840))

# Since we are training the full model now, we will combine the train and validation data to fit the model and only
# test on the test data - thus, we split these up into two separate lists of lag matrices
embed_train_val_list <- lapply(embed_mat_list,
                           function(x) rbind(data.frame(x$train), data.frame(x$val)))
 
embed_test_list <- lapply(embed_mat_list,
                          function(x) data.frame(x$test))   

In [16]:
full_model <- readRDS("Results/Pooled Regression Full/global_ar_model.RData")

In [17]:
# Make predicitons on the test data using this model
global_ar_full <- global_ar_least_squares_pred(full_model$model, embed_test_list, TRUE, TRUE)

In [18]:
# Create a data frame of the test data
test_data_full <- do.call("rbind", embed_test_list)

In [19]:
# Create bootstrap pred intervals on the test data, using a bootstrap sample number of n_samp=1000
# We could use a smaller n_samp for speed, but higher will give us a more accurate picture of the PIs
n_samp <- 1000
boot_test <- bootstrap_pred_int(global_ar_full$test_preds,
                                full_model$train_residuals,
                                n_samp,
                                test_data_full[,1])     

In [20]:
# Examine the model performance - it is quite similar to the training data, which is good
mean(global_ar_full$test_rmse)

[1] 43.07633

In [21]:
mean(global_ar_full$test_mae)

[1] 28.17094

In [22]:
mean(global_ar_full$test_rmse/global_ar_full$test_means)

[1] 0.1239873

In [23]:
mean(global_ar_full$test_mae/global_ar_full$test_means)

[1] 0.08545497

In [24]:
mean((boot_test %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80)

[1] 180.8483

In [29]:
mean((boot_test %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80/
global_ar_full$test_means)

[1] 0.5202134

In [25]:
mean((boot_test %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95)

[1] 380.9605

In [30]:
mean((boot_test %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95/
global_ar_full$test_means)

[1] 1.017136

# Global AR Model for Highway System

In [31]:
highways_england_model <- readRDS("Results/Pooled Regression Clustering/Country/global_ar_model_country_1.RData")

In [32]:
# Make predicitons on the test data using this model
global_ar_england_pred <- global_ar_least_squares_pred(highways_england_model$model, embed_test_list, TRUE, TRUE)

In [33]:
# Create bootstrap pred intervals on the test data, using a bootstrap sample number of n_samp=1000
# We could use a smaller n_samp for speed, but higher will give us a more accurate picture of the PIs
n_samp <- 1000
boot_test_england <- bootstrap_pred_int(global_ar_england_pred$test_preds,
                                        highways_england_model$train_residuals,
                                        n_samp,
                                        test_data_full[,1])    

In [34]:
# Examine the model performance - it is quite similar to the training data, which is good
mean(global_ar_england_pred$test_rmse)

[1] 43.07012

In [35]:
mean(global_ar_england_pred$test_mae)

[1] 28.20053

In [36]:
mean(global_ar_england_pred$test_rmse/global_ar_england_pred$test_means)

[1] 0.1241873

In [37]:
mean(global_ar_england_pred$test_mae/global_ar_england_pred$test_means)

[1] 0.08565014

In [38]:
mean((boot_test_england %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80)

[1] 178.2301

In [68]:
mean((boot_test_england %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80/
global_ar_england_pred$test_means)

[1] 0.5432133

In [69]:
mean((boot_test_england %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95)

[1] 367.0306

In [70]:
mean((boot_test_england %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95/
global_ar_england_pred$test_means)

[1] 1.079283

# Global AR Random Clustering

In [42]:
clust_assign$rand

[1] 3 2 1 4

In [43]:
rand_clust_mod3 <- readRDS("Results/Pooled Regression Clustering/Random Clustering/global_ar_model_cluster_3.RData")
rand_clust_mod2 <- readRDS("Results/Pooled Regression Clustering/Random Clustering/global_ar_model_cluster_2.RData")
rand_clust_mod1 <- readRDS("Results/Pooled Regression Clustering/Random Clustering/global_ar_model_cluster_1.RData")
rand_clust_mod4 <- readRDS("Results/Pooled Regression Clustering/Random Clustering/global_ar_model_cluster_4.RData")

In [44]:
# Make predicitons on the test data using this model
test_pred_clust3 <- global_ar_least_squares_pred(rand_clust_mod3$model, embed_test_list[1], TRUE, TRUE)

test_pred_clust2 <- global_ar_least_squares_pred(rand_clust_mod2$model, embed_test_list[2], TRUE, TRUE)

test_pred_clust1 <- global_ar_least_squares_pred(rand_clust_mod2$model, embed_test_list[3], TRUE, TRUE)

test_pred_clust4 <- global_ar_least_squares_pred(rand_clust_mod2$model, embed_test_list[4], TRUE, TRUE)

In [45]:
# Create bootstrap pred intervals on the test data, using a bootstrap sample number of n_samp=1000
# We could use a smaller n_samp for speed, but higher will give us a more accurate picture of the PIs
n_samp <- 1000
boot_testclust3 <- bootstrap_pred_int(test_pred_clust3$test_preds,
                                      rand_clust_mod3$train_residuals,
                                      n_samp,
                                      embed_test_list[[1]][,1]) 

boot_testclust2 <- bootstrap_pred_int(test_pred_clust2$test_preds,
                                      rand_clust_mod2$train_residuals,
                                      n_samp,
                                      embed_test_list[[2]][,1]) 

boot_testclust1 <- bootstrap_pred_int(test_pred_clust1$test_preds,
                                      rand_clust_mod1$train_residuals,
                                      n_samp,
                                      embed_test_list[[3]][,1]) 

boot_testclust4 <- bootstrap_pred_int(test_pred_clust4$test_preds,
                                      rand_clust_mod4$train_residuals,
                                      n_samp,
                                      embed_test_list[[4]][,1]) 

In [46]:
mean(c(test_pred_clust3$test_rmse, test_pred_clust2$test_rmse, 
       test_pred_clust1$test_rmse, test_pred_clust4$test_rmse))

[1] 43.74631

In [47]:
mean(c(test_pred_clust3$test_mae, test_pred_clust2$test_mae,
       test_pred_clust1$test_mae, test_pred_clust4$test_mae))

[1] 28.86435

In [48]:
mean(c(test_pred_clust3$test_rmse, test_pred_clust2$test_rmse, test_pred_clust1$test_rmse, test_pred_clust4$test_rmse)/
     c(test_pred_clust3$test_means, test_pred_clust2$test_means, test_pred_clust1$test_means, test_pred_clust4$test_means))

[1] 0.1262033

In [49]:
mean(c(test_pred_clust3$test_mae, test_pred_clust2$test_mae, test_pred_clust1$test_mae, test_pred_clust4$test_mae)/
     c(test_pred_clust3$test_means, test_pred_clust2$test_means, test_pred_clust1$test_means, test_pred_clust4$test_means))

[1] 0.08753218

In [50]:
mean((rbind(boot_testclust3, boot_testclust2, boot_testclust1, boot_testclust4))$int_score_80)

[1] 182.9027

In [56]:
mean(c(mean(boot_testclust3$int_score_80), mean(boot_testclust2$int_score_80), 
mean(boot_testclust1$int_score_80), mean(boot_testclust4$int_score_80))/
c(test_pred_clust3$test_means, test_pred_clust2$test_means, 
  test_pred_clust1$test_means, test_pred_clust4$test_means))

[1] 0.5302007

In [57]:
mean((rbind(boot_testclust3, boot_testclust2, boot_testclust1, boot_testclust4))$int_score_95)

[1] 372.584

In [58]:
mean(c(mean(boot_testclust3$int_score_95), mean(boot_testclust2$int_score_95), 
mean(boot_testclust1$int_score_95), mean(boot_testclust4$int_score_95))/
c(test_pred_clust3$test_means, test_pred_clust2$test_means, 
  test_pred_clust1$test_means, test_pred_clust4$test_means))

[1] 1.016481

# Global AR Catch22

In [59]:
clust_assign$catch22

[1] 1 1 1 1

In [60]:
catch22_mod1 <- readRDS("Results/Pooled Regression Clustering/KMeans/global_ar_model_kmeans_catch22_clust_1.RData")

In [61]:
# Make predicitons on the test data using this model
test_pred_catch22_clust1 <- global_ar_least_squares_pred(catch22_mod1$model, 
                                                         embed_test_list, TRUE, TRUE)

In [62]:
# Create bootstrap pred intervals on the test data, using a bootstrap sample number of n_samp=1000
# We could use a smaller n_samp for speed, but higher will give us a more accurate picture of the PIs
n_samp <- 1000
boot_catch22_testclust1 <- bootstrap_pred_int(test_pred_catch22_clust1$test_preds,
                                              catch22_mod1$train_residuals,
                                              n_samp,
                                              test_data_full[,1])

In [63]:
mean(test_pred_catch22_clust1$test_rmse)

[1] 43.24717

In [64]:
mean(test_pred_catch22_clust1$test_mae)

[1] 28.30007

In [65]:
mean(test_pred_catch22_clust1$test_rmse/test_pred_catch22_clust1$test_means)

[1] 0.1244483

In [66]:
mean(test_pred_catch22_clust1$test_mae/test_pred_catch22_clust1$test_means)

[1] 0.08580028

In [67]:
mean((boot_catch22_testclust1 %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80)

[1] 180.3415

In [73]:
mean((boot_catch22_testclust1 %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80/
test_pred_catch22_clust1$test_means)

[1] 0.5277064

In [72]:
mean((boot_catch22_testclust1 %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95)

[1] 382.0754

In [71]:
mean((boot_catch22_testclust1 %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95/
test_pred_catch22_clust1$test_means)

[1] 1.025869

# GlobalAR TSFeat

In [74]:
clust_assign$tsfeat

[1] 2 2 2 1

In [75]:
tsfeat_mod2 <- readRDS("Results/Pooled Regression Clustering/KMeans/global_ar_model_kmeans_tsfeat_clust_2.RData")
tsfeat_mod1 <- readRDS("Results/Pooled Regression Clustering/KMeans/global_ar_model_kmeans_tsfeat_clust_1.RData")

In [76]:
# Make predicitons on the test data using this model
test_pred_tsfeat_clust2 <- global_ar_least_squares_pred(tsfeat_mod2$model, 
                                                        embed_test_list[1:3], TRUE, TRUE)

test_pred_tsfeat_clust1 <- global_ar_least_squares_pred(tsfeat_mod1$model, 
                                                        embed_test_list[4], TRUE, TRUE)

In [77]:
# Create bootstrap pred intervals on the test data, using a bootstrap sample number of n_samp=1000
# We could use a smaller n_samp for speed, but higher will give us a more accurate picture of the PIs
n_samp <- 1000
boot_tsfeat_testclust2 <- bootstrap_pred_int(test_pred_tsfeat_clust2$test_preds,
                                             tsfeat_mod2$train_residuals,
                                             n_samp,
                                             do.call(rbind, embed_test_list[1:3])[,1])

boot_tsfeat_testclust1 <- bootstrap_pred_int(test_pred_tsfeat_clust1$test_preds,
                                             tsfeat_mod1$train_residuals,
                                             n_samp,
                                             embed_test_list[[4]][,1])

In [78]:
mean(c(test_pred_tsfeat_clust2$test_rmse, test_pred_tsfeat_clust1$test_rmse))

[1] 43.18653

In [79]:
mean(c(test_pred_tsfeat_clust2$test_mae, test_pred_tsfeat_clust1$test_mae))

[1] 28.2579

In [80]:
mean(c(test_pred_tsfeat_clust2$test_rmse, test_pred_tsfeat_clust1$test_rmse)/
    c(test_pred_tsfeat_clust2$test_means, test_pred_tsfeat_clust1$test_means))

[1] 0.1241995

In [81]:
mean(c(test_pred_tsfeat_clust2$test_mae, test_pred_tsfeat_clust1$test_mae)/
    c(test_pred_tsfeat_clust2$test_means, test_pred_tsfeat_clust1$test_means))

[1] 0.08558247

In [82]:
mean(rbind(boot_tsfeat_testclust2, boot_tsfeat_testclust1)$int_score_80)

[1] 180.1195

In [89]:
mean(c((boot_tsfeat_testclust2 %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80,
(boot_tsfeat_testclust1 %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80)/
c(test_pred_tsfeat_clust2$test_means, test_pred_tsfeat_clust1$test_means))

[1] 0.5218113

In [90]:
mean(rbind(boot_tsfeat_testclust2, boot_tsfeat_testclust1)$int_score_95)

[1] 367.0933

In [91]:
mean(c((boot_tsfeat_testclust2 %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95,
(boot_tsfeat_testclust1 %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95)/
c(test_pred_tsfeat_clust2$test_means, test_pred_tsfeat_clust1$test_means))

[1] 0.9957546

# GlobalAR DTW

In [92]:
clust_assign$dtw

[1] 2 1 1 1

In [93]:
dtw_clust_mod2 <- readRDS("Results/Pooled Regression Clustering/DTW/global_ar_dtw_clust_2.RData")
dtw_clust_mod1 <- readRDS("Results/Pooled Regression Clustering/DTW/global_ar_dtw_clust_1.RData")

In [94]:
# Make predicitons on the test data using this model
# Make predicitons on the test data using this model
test_pred_dtw_clust2 <- global_ar_least_squares_pred(dtw_clust_mod2$model,
                                                     embed_test_list[1], TRUE, TRUE)

test_pred_dtw_clust1 <- global_ar_least_squares_pred(dtw_clust_mod1$model,
                                                     embed_test_list[2:4], TRUE, TRUE)

In [95]:
# Create bootstrap pred intervals on the test data, using a bootstrap sample number of n_samp=1000
# We could use a smaller n_samp for speed, but higher will give us a more accurate picture of the PIs
n_samp <- 1000

boot_dtwclust_2 <- bootstrap_pred_int(test_pred_dtw_clust2$test_preds,
                                      dtw_clust_mod2$train_residuals,
                                      n_samp,
                                      embed_test_list[[1]][,1]) 


boot_dtwclust_1 <- bootstrap_pred_int(test_pred_dtw_clust1$test_preds,
                                      dtw_clust_mod1$train_residuals,
                                      n_samp,
                                      do.call(rbind, embed_test_list[2:4])[,1]) 

In [96]:
mean(c(test_pred_dtw_clust1$test_rmse, test_pred_dtw_clust2$test_rmse))

[1] 42.9348

In [97]:
mean(c(test_pred_dtw_clust1$test_mae, test_pred_dtw_clust2$test_mae))

[1] 28.16002

In [98]:
mean(c(test_pred_dtw_clust1$test_rmse, test_pred_dtw_clust2$test_rmse)/
     c(test_pred_dtw_clust1$test_means, test_pred_dtw_clust2$test_means)
    )

[1] 0.1236922

In [99]:
mean(c(test_pred_dtw_clust1$test_mae, test_pred_dtw_clust2$test_mae)/
     c(test_pred_dtw_clust1$test_means, test_pred_dtw_clust2$test_means)
    )

[1] 0.08556851

In [100]:
mean((rbind(boot_dtwclust_1, boot_dtwclust_2))$int_score_80)

[1] 175.027

In [101]:
mean(c((boot_dtwclust_1 %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80,
(boot_dtwclust_2 %>% group_by(ts_index) %>% summarise(int_score_80=mean(int_score_80)))$int_score_80)/
c(test_pred_dtw_clust1$test_means, test_pred_dtw_clust2$test_means))

[1] 0.5926542

In [102]:
mean((rbind(boot_dtwclust_1, boot_dtwclust_2))$int_score_95)

[1] 337.1282

In [103]:
mean(c((boot_dtwclust_1 %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95,
(boot_dtwclust_2 %>% group_by(ts_index) %>% summarise(int_score_95=mean(int_score_95)))$int_score_95)/
c(test_pred_dtw_clust1$test_means, test_pred_dtw_clust2$test_means))

[1] 1.105711